## Hypothesis Test and Estimate d_xy

In this file, we are using the CDE methods proposed in the paper to detect the existance of contrastive dimension and estimate the contrastvie dimension value (d_xy).

In [1]:
import numpy as np
import pandas as pd

In [2]:
background_control = pd.read_csv("background_control.csv")
foreground_Etop = pd.read_csv("foreground_Etop.csv")
foreground_H2O2 = pd.read_csv("foreground_H2O2.csv")
foreground_Starve = pd.read_csv("foreground_Starve.csv")

print("background shape: ", background_control.shape)
print("foreground_Etop shape: ", foreground_Etop.shape)
print("foreground_H2O2 shape: ", foreground_H2O2.shape)
print("foreground_Starve shape: ", foreground_Starve.shape)

background shape:  (11268, 47)
foreground_Etop shape:  (4315, 47)
foreground_H2O2 shape:  (5015, 47)
foreground_Starve shape:  (3007, 47)


### CDE Methods

In [3]:
import numpy as np
import pandas as pd
import skdim
from scipy.linalg import eigh
from sklearn.preprocessing import scale


def id_estimators(df, k):
    # Maximum Likelihood algorithm
    MLE = skdim.id.MLE(K=k).fit(df).dimension_
    # Method Of Moments algorithm
    MOM = skdim.id.MOM().fit(df).dimension_
    L = {
        'MLE': MLE,
        'MOM': MOM,
    }
    return L


def est_V1_V2(X1, X2, d1, d2):
    OUT = {}
    p = X1.shape[1]
    Cx1 = np.cov(X1, rowvar=False)
    Cx2 = np.cov(X2, rowvar=False)
    # eigenvalues python package in increasing order
    val1, vectors1 = eigh(Cx1)
    idx = np.argsort(val1)
    descending_idx = idx[::-1]
    vectors1 = vectors1[:, descending_idx]
    V1 = vectors1[:, 0:d1]
    val2, vectors2 = eigh(Cx2)
    idx_ = np.argsort(val2)
    descending_idx_ = idx_[::-1]
    vectors2 = vectors2[:, descending_idx_]
    V2 = vectors2[:, 0:d2]
    OUT['V1'] = V1
    OUT['V2'] = V2
    return OUT


def sigma1_test_stat(X1, X2, d1, d2):
    OUT = est_V1_V2(X1, X2, d1, d2)
    U = OUT['V1']
    V = OUT['V2']
    M = np.matmul(U.T, V)
    _, cosines, _ = np.linalg.svd(M)
    cosines = np.minimum(1, np.maximum(-1, cosines))
    return cosines[::-1][0]     # first elt of reversed cosines list


def sing_vals(U, V):
    M = np.matmul(U.T, V)
    _, cosines, _ = np.linalg.svd(M)
    cosines = np.minimum(1, np.maximum(-1, cosines))
    return cosines


def boot_test(X1, X2, d1, d2, B):
    X1 = scale(X1, with_mean=True, with_std=False)
    X2 = scale(X2, with_mean=True, with_std=False)
    test_stat = sigma1_test_stat(X1, X2, d1, d2)
    n1 = len(X1)
    n2 = len(X2)
    boot_stats = []
    for j in range(1, B+1):
        # print(j)
        idx1 = np.random.choice(range(n1), size=n1, replace=True)
        X1t = X1[idx1, :]
        combined = np.vstack((X1, X2))
        idx2 = np.random.choice(range(n1+n2), size=n2, replace=True)
        X2t = combined[idx2, :]
        boot_stats.append(sigma1_test_stat(X1t, X2t, d1, d2))
    p_value = np.mean(boot_stats < test_stat)
    return {'test_stat': test_stat, 'p_value': p_value}


def CD(X1, X2, d1, d2, epsilon=0.1, B=1000):
    p = X1.shape[1]
    OUT = est_V1_V2(X1, X2, d1, d2)
    singular_vals = sing_vals(OUT['V1'], OUT['V2'])
    singular_vals = singular_vals[::-1]
    L = {}
    L['CD'] = sum(singular_vals < 1 - epsilon) + max(d1 - d2, 0)
    test = boot_test(X1, X2, d1, d2, B)
    L['test_stat'] = test['test_stat']
    L['p_value'] = test['p_value']
    L['singular_vals'] = singular_vals
    L['d1'] = d1
    L['d2'] = d2
    return L


def CDE(fg, bg):
    L1 = id_estimators(fg, 10)
    d1 = round(L1["MOM"])
    L2 = id_estimators(bg, 10)
    d2 = round(L2["MOM"])
    return CD(fg, bg, d1, d2)

In [4]:
# Etop
np.random.seed(42)
CDE(foreground_Etop, background_control)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f6a0a269990>
Traceback (most recent call last):
  File "/nas/longleaf/rhel8/apps/anaconda/2023.03.ood/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/nas/longleaf/rhel8/apps/anaconda/2023.03.ood/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/nas/longleaf/rhel8/apps/anaconda/2023.03.ood/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/nas/longleaf/rhel8/apps/anaconda/2023.03.ood/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling cty

{'CD': 8,
 'test_stat': 0.11824350122520415,
 'p_value': 0.0,
 'singular_vals': array([0.1182435 , 0.16808648, 0.42584698, 0.48392107, 0.58180365,
        0.68918889, 0.80683974, 0.89514646, 0.92018272, 0.98105305]),
 'd1': 10,
 'd2': 12}

In [5]:
# H2O2
np.random.seed(42)
CDE(foreground_H2O2, background_control)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f69ca631ea0>
Traceback (most recent call last):
  File "/nas/longleaf/rhel8/apps/anaconda/2023.03.ood/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/nas/longleaf/rhel8/apps/anaconda/2023.03.ood/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/nas/longleaf/rhel8/apps/anaconda/2023.03.ood/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/nas/longleaf/rhel8/apps/anaconda/2023.03.ood/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling cty

{'CD': 7,
 'test_stat': 0.220255118354846,
 'p_value': 0.0,
 'singular_vals': array([0.22025512, 0.37392305, 0.59035934, 0.64095297, 0.75493582,
        0.87317152, 0.8962471 , 0.94559443, 0.96174109, 0.9825771 ]),
 'd1': 10,
 'd2': 12}

In [6]:
# Starve 
np.random.seed(42)
CDE(foreground_Starve, background_control)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f69ca631f30>
Traceback (most recent call last):
  File "/nas/longleaf/rhel8/apps/anaconda/2023.03.ood/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/nas/longleaf/rhel8/apps/anaconda/2023.03.ood/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/nas/longleaf/rhel8/apps/anaconda/2023.03.ood/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/nas/longleaf/rhel8/apps/anaconda/2023.03.ood/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling cty

{'CD': 5,
 'test_stat': 0.3003133950308991,
 'p_value': 0.355,
 'singular_vals': array([0.3003134 , 0.39087243, 0.47791836, 0.79047855, 0.81259976,
        0.90037128, 0.9620981 , 0.96851792, 0.98545592]),
 'd1': 9,
 'd2': 12}